# Choosing a Class Variable
When predicting the stability of a compound, there are a few properties that can be used interchangably to assess stability. One option is to first is to predict the formation enthalpy of the material, and then compute the stabilty by measuring the difference between that value and the formation enthalpies of all other known compounds. Or, you could predict the stability directly. In this notebook, we compare the two approaches

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from scipy.stats import kendalltau, bayes_mvs
import numpy as np
import pandas as pd
import json
import os

/home/wardlt/software/scikit-learn/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Key variables

In [2]:
n_repeats = 25 # Number of times you repeat the CV test
n_steps = 8 # Number of sizes of QH dataset to use for training set
test_fraction = 0.1 # Fraction of QH dataset to withhold as test set

## Load in the Data
Read in the data from JSON to get the training and test sets

In [3]:
def load_data(path):
    """Read in a JSON file from disk, and return the inputs and outputs"""
    
    temp = json.load(open(path))
    de_id = [x['name'] for x in temp['properties']].index('delta_e')
    X = np.array([x['attributes'] for x in temp['entries']], dtype=np.float32)
    stability = np.array([x['class']['measured'] for x in temp['entries']], dtype=np.float32)
    delta_e = np.array([x['properties'][de_id]['measured'] for x in temp['entries']], dtype=np.float32)
    return X, {'stability': stability, 'delta_e': delta_e}

Load in the Quaternary Heusler dataset

In [4]:
qh_X, qh_y = load_data(os.path.join('..', 'datasets', 'quat-heuslers.json'))
print('Read in %d QH entries'%len(qh_X))

Read in 96188 QH entries


Load in the ternary Heusler dataset

In [5]:
th_X, th_y = load_data(os.path.join('..', 'datasets', 'heuslers.json'))
print('Read in %d ternary Heusler entries'%len(th_X))

Read in 184094 ternary Heusler entries


Load in everything else from the OQMD

In [6]:
oqmd_X, oqmd_y= load_data(os.path.join('..', 'datasets', 'oqmd-no-heusler.json'))
print('Read in %d entries from the rest of the OQMD'%len(oqmd_X))

Read in 145866 entries from the rest of the OQMD


## Run the Test
Our test is basically a random split cross-validation test. We first split off a chunk of the QH dataset to use as a test set. Then, we train a model on the remaining QH data and the rest of the OQMD. We then test the performance when using formation enthalpy and stability as the output variable. We repeat this test multiple times with different QH hold-out sets.

In [7]:
model = Pipeline([
    ('imputer', Imputer()),
    ('rf', RandomForestRegressor(n_estimators=100, n_jobs=-1)),
])

In [8]:
test_size = int(test_fraction * len(qh_X))

In [9]:
def run_test(train_X, train_Y, test_X, test_Y, label):
    model.fit(train_X, train_Y)
    pred_Y = model.predict(test_X)
    R = metrics.r2_score(test_Y, pred_Y)
    MAE = metrics.mean_absolute_error(test_Y, pred_Y)
    MSE = metrics.mean_squared_error(test_Y, pred_Y)
    tau = kendalltau(test_Y, pred_Y)
    return {'label': label, 'R2': R, 
            'MAE': MAE, 'MSE': MSE, 'tau': tau.correlation}

In [10]:
results = [] # Holds dictionaries with each result

In [11]:
for rep in tqdm(range(n_repeats), desc='Repeat'):
    for l in ['delta_e', 'stability']:
        # Get the hold out set for the QHs
        train_X, test_X, train_y, test_y = train_test_split(qh_X, qh_y[l], test_size=test_size, random_state=rep)

        # Append the OQMD and TH training data
        train_X = np.vstack([train_X, oqmd_X])
        train_X = np.vstack([train_X, th_X])

        train_y = np.hstack([train_y, oqmd_y[l]])
        train_y = np.hstack([train_y, th_y[l]])

        # Run the test
        results.append(run_test(train_X, train_y, test_X, test_y, l))

Widget Javascript not detected.  It may not be installed or enabled properly.


In [12]:
test_results = pd.DataFrame(results)

In [13]:
test_results.to_csv('test_results.csv', index=False)